In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pickle
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm_notebook as tqdm

from nlp_surveillance.scraper import who_scraper, promed_scraper, text_extractor

In [ ]:
who_urls = who_scraper.scrape()

In [ ]:
who_urls = who_urls.URL.tolist()

In [ ]:
with open('who_urls.pkl', 'wb') as f:
    pickle.dump(who_urls, f)

In [84]:
with open('who_urls.pkl', 'rb') as f:
    who_urls = pickle.load(f)

In [ ]:
len(who_urls)

In [ ]:
promed_urls = promed_scraper.scrape(from_date='01/01/1994', to_date='31/12/2018')

In [ ]:
promed_urls = promed_urls.URL.tolist()

In [77]:
with open('promed_urls.pkl', 'wb') as f:
    pickle.dump(promed_urls.URL.tolist(), f)

In [ ]:
len(promed_urls)

In [85]:
all_urls = promed_urls
all_urls.extend(who_urls)

In [ ]:
for url in tqdm(all_urls):
    text = text_extractor.extract_cleaned_text_from_url(url)
    path_name = f"data/corpus/{url.replace('/', '_')}.txt"
    with open(path_name, 'w') as f:
        f.write(text)     